## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Camera Calibration

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## Apply Undistortion

In [6]:
def display_img(img):
    cv2.imshow("test", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def warp_binary_img(img, sobelx_thresh_min = 40, sobelx_thresh_max = 100, s_thresh_min=170, s_thresh_max = 255):
    # undistort image
    undistort = cv2.undistort(img, mtx, dist, None, mtx)
    # grayscale image
    gray = cv2.cvtColor(undistort, cv2.COLOR_BGR2GRAY)
    
    #sobel x
    sobelx = np.abs(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    scaled_sobel = np.uint8(255*sobelx/np.max(sobelx))
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sobelx_thresh_min) & (scaled_sobel <= sobelx_thresh_max)] = 1
    
    # threshold color channel
    hls = cv2.cvtColor(undistort, cv2.COLOR_BGR2HLS)
    s_channel = hls[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    # combining thresholded images
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    # create binary image
    binary_image = combined_binary * 255
    # create perspective transform
    src = np.float32([[535, 500], [790, 500], [1105, 700], [280, 700]])
    dst = np.float32([[200, 0], [1000, 0], [1000, 680], [200, 680]])
    M = cv2.getPerspectiveTransform(src, dst)
    warped_binary = cv2.warpPerspective(binary_image, M, binary_image.shape[::-1], flags=cv2.INTER_LINEAR)
    
    return warped_binary

In [73]:
def find_lane_pixels(binary_img, nwindows=9, margin=100, minpix=50):
    # create histogram of bottom half of the image
    histogram = np.sum(2*binary_img[binary_img.shape[0]//3:, :], axis=0)
    midpoint = histogram.shape[0]//2
    # initial guess for starting point of left and right lanes
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    window_height = binary_img.shape[0]//nwindows
    
    # identify x and y position of all nonzero pixels in image
    nonzero = binary_img.nonzero()
    nonzeroy = nonzero[0]
    nonzerox = nonzero[1]
    
    # current positions to be updated late for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    out_img = np.dstack((binary_img, binary_img, binary_img))
    for window in range(nwindows):
        # identify window boundaries in x and y
        win_y_low = binary_img.shape[0] - (window+1)*window_height
        win_y_high = binary_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(int(win_xleft_low),int(win_y_low)),(int(win_xleft_high),int(win_y_high)),(0,255,0), 2) 
        cv2.rectangle(out_img,(int(win_xright_low),int(win_y_low)),(int(win_xright_high),int(win_y_high)),(0,255,0), 2) 
        
        # identify nonzero pixels in x and y within window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # append indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # if found enought good indicies, recenter next window on their mean position 
        if len(good_left_inds) > minpix:
            leftx_current = np.mean(nonzerox[good_left_inds])
        if len(good_right_inds) > minpix:
            rightx_current = np.mean(nonzerox[good_right_inds])
            
    # concatenate the arrays of indices
 
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_img):
    # find lane pixels
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_img)
    
    # fit a second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # generate x and y values for plotting
    ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0])
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.imshow(out_img)

In [78]:
test_img = cv2.imread("./test_images/test4.jpg")
warped_binary = warp_binary_img(test_img)
fit_polynomial(warped_binary)